In [39]:
'''
   Name: Seema Goel
   Date : 22 April 2018
   version 1.0.0
   Purpose: To load the videos,extract suitable features and train a Machine Learning model to classify each video as walking or jogging.

'''


import os
import sys
import argparse
import csv
import numpy as np
import imageio
imageio.plugins.ffmpeg.download()
from random import randint
from PIL import Image

In [40]:
'''
    identify_classes  is the function to figure out lables in the dataset
    VARIABLES:
     path : path to dataset folder
     classes : list of classes to be classified
     l_folders: classes folder in train dataset
     labels : are classes/lables to be identifed
     below function is identifing lables/classes 
'''
def identify_classes(path):
    # empty list of lables
    classes = []
    
    l_folders  = os.listdir(path)
    index = 0
    
    # looping over Directory structure to extract folders name as classes
    for labels in sorted(l_folders):
        label_folder_path = os.path.join(path, labels)
        if os.path.isdir(label_folder_path):
            group_folders = os.listdir(label_folder_path)
            for group_folder in group_folders:
                # no annototions required here 
                if group_folder != 'Annotation':
                    classes.append([os.path.join(label_folder_path, group_folder), index])
                    index += 1

    return classes

In [41]:
'''
  video_Files_split is splitiing the dataset into training and test with 80% and 20%
 Variables:
 group_count : number of classes/lables
  indices: holds evenly spaced values
  train_count: hold number of files in training set
  test_count: number of files in test set
  test, train: list to hold name of file in both sets
  video_files = list containing video files
'''

def video_Files_split(classes, file_ext):
    
    group_count = len(classes)
    indices = np.arange(group_count)

    np.random.seed(0) # Make it deterministic.
    np.random.shuffle(indices)

    # 80% training and 20% test.
    train_count = int(0.8 * group_count)
    test_count  = group_count - train_count

    train = [] # this will hold the name of files in train set
    test  = [] # this will hold 
    # looping over videos to split videos files into training list 
    for i in range(train_count):
        group = classes[indices[i]]
        print("Current category",group)
        video_files = os.listdir(group[0])
        for video_file in video_files:
            video_file_path = os.path.join(group[0], video_file)
            if os.path.isfile(video_file_path):
                video_file_path = os.path.abspath(video_file_path)
                ext = os.path.splitext(video_file_path)[1]
                if (ext == file_ext):
                    # reading only files with .avi extension and creating frames
                    video_reader = imageio.get_reader(video_file_path, 'ffmpeg')                    
                    if len(video_reader) >= 16:
                        train.append([video_file_path, group[1]])
                        
                        
                        
    #looping over files to put refernce of files in training list 
    for i in range(train_count, train_count + test_count):
        group = classes[indices[i]]
        video_files = os.listdir(group[0])
        for video_file in video_files:
            video_file_path = os.path.join(group[0], video_file)
            if os.path.isfile(video_file_path):
                video_file_path = os.path.abspath(video_file_path)
                ext = os.path.splitext(video_file_path)[1]
                if (ext == file_ext):
                    # # reading only files with .avi extension and creating frames
                    video_reader = imageio.get_reader(video_file_path, 'ffmpeg')
                    if len(video_reader) >= 16:
                        test.append([video_file_path, group[1]])

    return train, test

In [42]:
'''
    Path file names are written in CSV files
    
'''

def create_csv(items, file_path):
   
    if sys.version_info[0] < 3:
        with open(file_path, 'wb') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for item in items:
                writer.writerow(item)
    else:
        with open(file_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for item in items:
                writer.writerow(item)
    print("thumbsup")

In [43]:
classes = identify_classes("data/")
len(classes)

2

In [44]:
train, test = video_Files_split(classes, '.avi')

Current category ['data/train/walking', 1]


In [45]:
len(train)

52

In [46]:
len(test)

52

In [47]:
create_csv(train, os.path.join('data/', 'train_map.csv'))
create_csv(test, os.path.join('data/', 'test_map.csv'))

thumbsup
thumbsup


#####  Reading video from path in csv, extract frames

##### Convert frames to numpy arrays and save in file


In [48]:
def select_features(video_file, sequence_length, is_training,width,height):
        '''
        Select a sequence of frames from video_file and return them as
        a Tensor.
        '''
        try:
            video_reader = imageio.get_reader(video_file, 'ffmpeg')
            num_frames   = len(video_reader)
            if sequence_length > num_frames:
                raise ValueError('Sequence length {} is larger then the total number of frames {} in {}.'.format(sequence_length, num_frames, video_file))

            # select which sequence frames to use.
            step = 1
            expanded_sequence = sequence_length
            if num_frames > 2*sequence_length:
                step = 2
                expanded_sequence = 2*sequence_length

            seq_start = int(num_frames/2) - int(expanded_sequence/2)
            if is_training:
                seq_start = randint(0, num_frames - expanded_sequence)

            frame_range = [seq_start + step*i for i in range(sequence_length)]            
            video_frames = []
            for frame_index in frame_range:
                video_frames.append(read_frame(video_reader.get_data(frame_index),width,height))
        
            return np.stack(video_frames, axis=1)
        except:
            print("Error reading video")

In [49]:
 '''
        Using best practices/augumentation of 3D CNN resizing the image to 128x171 first, then selecting a 112x112 crop.
 '''

def read_frame(data,width,height):
       
        try:
            if (width >= 171) or (height >= 128):
                raise ValueError("Target width need to be less than 171 and target height need to be less than 128.")
        
            image = Image.fromarray(data)
            image.thumbnail((171, 128), Image.ANTIALIAS)
        
            center_w = image.size[0] / 2
            center_h = image.size[1] / 2

            image = image.crop((center_w - width  / 2,
                                center_h - height / 2,
                                center_w + width  / 2,
                                center_h + height / 2))
        
            norm_image = np.array(image, dtype=np.float32)
            norm_image -= 127.5
            norm_image /= 127.5

            # (channel, height, width)
            return np.ascontiguousarray(np.transpose(norm_image, (2, 0, 1)))
        except:
            print("Error reading frame")

In [50]:
'''
        Load video file paths and their corresponding labels.
'''

def read_video(map_file, label_count, is_training, limit_epoch_size=sys.maxsize):
       
        width           = 112
        height          = 112
        sequence_length = 16
        channel_count   = 3
        is_training     = is_training
        video_files     = []
        targets         = []
        batch_start     = 0

        
        map_file_dir = os.path.dirname(map_file)

        with open(map_file) as csv_file:
                data = csv.reader(csv_file)
                for row in data:
                    video_files.append(os.path.join(map_file_dir, row[0]))
                    target = [0.0] * label_count
                    target[int(row[1])] = 1.0
                    targets.append(target)

        indices = np.arange(len(video_files))
        if is_training:
                np.random.shuffle(indices)
        epoch_size = min(len(video_files), limit_epoch_size)
        vectored_video_files = []
        for video_file in video_files:
            vectored_video_file = select_features(video_file, sequence_length, is_training,width,height)
            print(vectored_video_file)
            vectored_video_files.append(vectored_video_file)
            
        return vectored_video_files,targets
        

In [51]:
num_output_classes = 2
train_reader = read_video(os.path.join('data/', 'train_map.csv'), num_output_classes, True)
test_reader  = read_video(os.path.join('data/', 'test_map.csv'), num_output_classes, True)

[[[[0.8509804  0.8509804  0.8666667  ... 0.8901961  0.8901961
    0.8745098 ]
   [0.8509804  0.8509804  0.84313726 ... 0.8901961  0.8745098
    0.8666667 ]
   [0.81960785 0.81960785 0.8352941  ... 0.8980392  0.8901961
    0.8901961 ]
   ...
   [0.64705884 0.64705884 0.7411765  ... 0.69411767 0.8352941
    0.7490196 ]
   [0.73333335 0.8117647  0.70980394 ... 0.8352941  0.6156863
    0.6313726 ]
   [0.67058825 0.654902   0.6156863  ... 0.6156863  0.654902
    0.8745098 ]]

  [[0.8509804  0.8509804  0.85882354 ... 0.8745098  0.8745098
    0.8666667 ]
   [0.8509804  0.8509804  0.8509804  ... 0.8901961  0.8745098
    0.8745098 ]
   [0.81960785 0.81960785 0.8352941  ... 0.8901961  0.8745098
    0.8745098 ]
   ...
   [0.73333335 0.5921569  0.70980394 ... 0.81960785 0.7882353
    0.6784314 ]
   [0.78039217 0.7019608  0.6784314  ... 0.8745098  0.6862745
    0.64705884]
   [0.5764706  0.6862745  0.69411767 ... 0.654902   0.70980394
    0.7882353 ]]

  [[0.8509804  0.85882354 0.8666667  ... 0.890

[[[[0.7254902  0.7019608  0.67058825 ... 0.75686276 0.7647059
    0.78039217]
   [0.75686276 0.78039217 0.73333335 ... 0.7647059  0.7647059
    0.7490196 ]
   [0.70980394 0.7019608  0.7411765  ... 0.7411765  0.7490196
    0.75686276]
   ...
   [0.7254902  0.7254902  0.70980394 ... 0.75686276 0.79607844
    0.70980394]
   [0.6784314  0.6784314  0.6392157  ... 0.56078434 0.6
    0.69411767]
   [0.6862745  0.6784314  0.7019608  ... 0.6784314  0.7254902
    0.70980394]]

  [[0.70980394 0.7254902  0.6862745  ... 0.78039217 0.78039217
    0.7882353 ]
   [0.79607844 0.7647059  0.73333335 ... 0.7882353  0.78039217
    0.7647059 ]
   [0.7411765  0.70980394 0.7647059  ... 0.7647059  0.7647059
    0.78039217]
   ...
   [0.70980394 0.7490196  0.7254902  ... 0.78039217 0.8039216
    0.75686276]
   [0.67058825 0.67058825 0.6392157  ... 0.5764706  0.6
    0.70980394]
   [0.6862745  0.6862745  0.73333335 ... 0.6784314  0.70980394
    0.7490196 ]]

  [[0.7490196  0.73333335 0.6862745  ... 0.7882353  0.

[[[[ 0.09803922  0.15294118  0.19215687 ...  0.24705882  0.18431373
     0.24705882]
   [ 0.14509805  0.12941177  0.09803922 ...  0.1764706   0.1764706
     0.09803922]
   [ 0.12941177  0.07450981  0.14509805 ...  0.12941177  0.20784314
     0.19215687]
   ...
   [ 0.07450981  0.08235294  0.04313726 ...  0.09803922  0.09803922
     0.07450981]
   [-0.01960784 -0.02745098 -0.21568628 ... -0.03529412 -0.01176471
     0.03529412]
   [ 0.03529412 -0.05098039  0.01960784 ...  0.04313726  0.09803922
    -0.03529412]]

  [[ 0.14509805  0.09803922  0.16862746 ...  0.23137255  0.2
     0.19215687]
   [ 0.14509805  0.09803922  0.09019608 ...  0.18431373  0.16862746
     0.1764706 ]
   [ 0.09019608  0.09803922  0.14509805 ...  0.14509805  0.15294118
     0.2       ]
   ...
   [-0.03529412  0.01176471  0.01176471 ...  0.01176471  0.03529412
     0.15294118]
   [ 0.02745098  0.02745098  0.02745098 ...  0.01960784  0.00392157
    -0.06666667]
   [-0.03529412 -0.15294118  0.08235294 ...  0.05882353  

[[[[ 0.11372549  0.11372549  0.12156863 ...  0.18431373  0.19215687
     0.15294118]
   [ 0.14509805  0.12156863  0.1764706  ...  0.09803922  0.07450981
     0.08235294]
   [ 0.1764706   0.18431373  0.2        ...  0.4745098   0.5137255
     0.5137255 ]
   ...
   [-0.07450981  0.01176471  0.01960784 ...  0.03529412 -0.09019608
    -0.01960784]
   [-0.04313726 -0.01960784 -0.06666667 ... -0.01960784 -0.09019608
     0.00392157]
   [-0.02745098 -0.07450981 -0.09019608 ... -0.05098039  0.00392157
     0.03529412]]

  [[ 0.09803922  0.09803922  0.13725491 ...  0.18431373  0.1764706
     0.16862746]
   [ 0.15294118  0.12941177  0.12156863 ...  0.11372549  0.12156863
     0.09019608]
   [ 0.20784314  0.2         0.1764706  ...  0.39607844  0.46666667
     0.4509804 ]
   ...
   [-0.05098039  0.02745098  0.02745098 ...  0.02745098 -0.04313726
    -0.01960784]
   [-0.02745098 -0.04313726 -0.05098039 ... -0.02745098 -0.06666667
    -0.01176471]
   [-0.02745098 -0.05098039 -0.07450981 ... -0.0745

[[[[ 0.12156863  0.11372549  0.12156863 ...  0.13725491  0.11372549
     0.14509805]
   [ 0.08235294  0.09019608  0.12941177 ...  0.16862746  0.14509805
     0.14509805]
   [ 0.12156863  0.12941177  0.14509805 ...  0.12941177  0.12941177
     0.14509805]
   ...
   [ 0.04313726 -0.04313726  0.00392157 ... -0.08235294 -0.09803922
    -0.03529412]
   [-0.03529412 -0.06666667 -0.07450981 ... -0.14509805 -0.09803922
    -0.10588235]
   [-0.07450981 -0.04313726 -0.07450981 ... -0.10588235 -0.06666667
    -0.08235294]]

  [[ 0.11372549  0.11372549  0.11372549 ...  0.12941177  0.12941177
     0.12941177]
   [ 0.09019608  0.11372549  0.12941177 ...  0.15294118  0.14509805
     0.14509805]
   [ 0.12156863  0.09803922  0.14509805 ...  0.12941177  0.12941177
     0.12941177]
   ...
   [ 0.00392157 -0.06666667 -0.07450981 ... -0.06666667 -0.10588235
    -0.03529412]
   [-0.07450981 -0.06666667 -0.07450981 ... -0.15294118 -0.09019608
    -0.16078432]
   [-0.09019608 -0.04313726 -0.09019608 ... -0.12

...

[[[[ 0.01960784  0.01960784  0.03529412 ...  0.11372549  0.09019608
     0.08235294]
   [ 0.03529412  0.01176471  0.01960784 ...  0.05882353  0.01176471
     0.03529412]
   [ 0.02745098  0.01960784  0.00392157 ...  0.00392157 -0.01960784
    -0.01176471]
   ...
   [ 0.00392157  0.01176471 -0.01960784 ... -0.05098039 -0.04313726
    -0.01960784]
   [-0.01176471 -0.01176471 -0.03529412 ... -0.05098039 -0.02745098
    -0.02745098]
   [-0.03529412 -0.02745098  0.01176471 ... -0.05098039 -0.04313726
    -0.06666667]]

  [[ 0.02745098  0.02745098  0.04313726 ...  0.12156863  0.08235294
     0.07450981]
   [ 0.04313726  0.04313726  0.02745098 ...  0.04313726  0.01960784
     0.03529412]
   [ 0.01960784  0.01960784  0.01176471 ...  0.00392157 -0.01176471
    -0.01960784]
   ...
   [-0.01960784 -0.01176471 -0.01176471 ... -0.06666667 -0.04313726
    -0.02745098]
   [-0.03529412 -0.03529412 -0.03529412 ... -0.06666667 -0.05098039
    -0.03529412]
   [-0.01960784 -0.01960784 -0.01176471 ... -0.05

[[[[ 0.22352941  0.2         0.19215687 ...  0.02745098  0.07450981
     0.07450981]
   [ 0.1764706   0.20784314  0.2        ...  0.00392157  0.09019608
     0.01960784]
   [ 0.19215687  0.25490198  0.13725491 ...  0.13725491  0.12941177
     0.08235294]
   ...
   [-0.07450981  0.03529412 -0.07450981 ...  0.00392157  0.11372549
     0.08235294]
   [-0.05098039 -0.07450981  0.02745098 ... -0.04313726  0.06666667
    -0.05098039]
   [-0.01176471  0.00392157  0.07450981 ...  0.01960784 -0.08235294
    -0.03529412]]

  [[ 0.19215687  0.2         0.19215687 ...  0.00392157  0.09803922
     0.06666667]
   [ 0.20784314  0.23137255  0.20784314 ...  0.04313726  0.08235294
     0.03529412]
   [ 0.18431373  0.24705882  0.12941177 ...  0.13725491  0.14509805
     0.07450981]
   ...
   [-0.09019608  0.04313726 -0.06666667 ...  0.05882353  0.08235294
     0.11372549]
   [-0.07450981 -0.02745098 -0.01176471 ... -0.01960784  0.05882353
    -0.04313726]
   [-0.01960784  0.04313726  0.12156863 ... -0.01

[[[[ 0.11372549  0.07450981  0.13725491 ...  0.03529412  0.06666667
     0.09019608]
   [ 0.09019608  0.04313726  0.12941177 ...  0.07450981  0.05882353
     0.14509805]
   [ 0.05882353  0.05882353  0.09019608 ...  0.02745098  0.01176471
     0.04313726]
   ...
   [ 0.05882353  0.08235294 -0.06666667 ... -0.09803922 -0.02745098
    -0.1764706 ]
   [-0.01960784 -0.01176471  0.03529412 ... -0.07450981 -0.07450981
    -0.01960784]
   [-0.04313726 -0.05098039  0.00392157 ... -0.01960784 -0.08235294
    -0.04313726]]

  [[ 0.09019608  0.08235294  0.12941177 ...  0.03529412  0.05882353
     0.09019608]
   [ 0.08235294  0.03529412  0.11372549 ...  0.05882353  0.05882353
     0.13725491]
   [ 0.09019608  0.05882353  0.07450981 ...  0.00392157  0.01960784
     0.06666667]
   ...
   [ 0.01960784  0.07450981 -0.06666667 ... -0.08235294 -0.04313726
    -0.16078432]
   [-0.01960784 -0.01960784  0.04313726 ... -0.05098039 -0.07450981
    -0.04313726]
   [-0.05098039 -0.06666667  0.01176471 ... -0.01

[[[[ 0.18431373  0.04313726  0.13725491 ...  0.15294118  0.13725491
     0.11372549]
   [ 0.14509805  0.18431373  0.12156863 ...  0.11372549  0.16862746
     0.12156863]
   [ 0.19215687  0.28627452  0.2        ...  0.09019608  0.13725491
     0.18431373]
   ...
   [ 0.00392157 -0.01176471 -0.05098039 ... -0.18431373 -0.02745098
     0.15294118]
   [ 0.05882353  0.09019608 -0.05098039 ... -0.25490198 -0.02745098
    -0.12941177]
   [ 0.12941177 -0.07450981 -0.08235294 ... -0.03529412 -0.21568628
     0.04313726]]

  [[ 0.20784314  0.06666667  0.15294118 ...  0.12941177  0.14509805
     0.09803922]
   [ 0.12941177  0.19215687  0.1764706  ...  0.12156863  0.15294118
     0.12941177]
   [ 0.20784314  0.2627451   0.19215687 ...  0.07450981  0.11372549
     0.15294118]
   ...
   [ 0.00392157 -0.01176471 -0.04313726 ... -0.23137255 -0.02745098
     0.16862746]
   [-0.01176471  0.09803922 -0.05098039 ... -0.24705882  0.01176471
    -0.16078432]
   [ 0.09803922 -0.06666667 -0.06666667 ... -0.03

[[[[-0.40392157 -0.3882353  -0.38039216 ... -0.33333334 -0.3254902
    -0.33333334]
   [-0.40392157 -0.38039216 -0.38039216 ... -0.3019608  -0.3254902
    -0.33333334]
   [-0.38039216 -0.3647059  -0.37254903 ... -0.30980393 -0.28627452
    -0.3019608 ]
   ...
   [-0.5294118  -0.54509807 -0.5372549  ... -0.3647059  -0.35686275
    -0.35686275]
   [-0.54509807 -0.5529412  -0.5529412  ... -0.3254902  -0.31764707
    -0.3647059 ]
   [-0.54509807 -0.5529412  -0.54509807 ... -0.30980393 -0.30980393
    -0.33333334]]

  [[-0.4117647  -0.3882353  -0.37254903 ... -0.31764707 -0.33333334
    -0.31764707]
   [-0.40392157 -0.3882353  -0.3882353  ... -0.30980393 -0.30980393
    -0.31764707]
   [-0.38039216 -0.3882353  -0.3647059  ... -0.3019608  -0.30980393
    -0.31764707]
   ...
   [-0.54509807 -0.5372549  -0.5529412  ... -0.30980393 -0.34901962
    -0.34901962]
   [-0.54509807 -0.5372549  -0.5372549  ... -0.3019608  -0.3254902
    -0.33333334]
   [-0.5686275  -0.5686275  -0.5529412  ... -0.31764

In [52]:
import numpy
numpy.save('data/train_data.npy', train_reader[0])

In [53]:
numpy.save('data/train_label.npy', train_reader[1])

In [54]:
numpy.save('data/test_data.npy', test_reader[0])

In [55]:
numpy.save('data/test_label.npy', test_reader[1])

In [56]:
data_npfile = np.load('data/test_label.npy')

In [57]:
data_npfile

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])